# Experiment

In [146]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from haversine import haversine
from sklearn.cluster import KMeans
import scipy.stats as stats
import folium
import itertools
import statistics
import string
import timeit
import random

import warnings
warnings.filterwarnings("ignore")

## Algorithm

> 주의사항

- 위도, 경도 수치로 변경해야함

### Naver Map API

In [147]:
# *-- Geocoding 활용 코드 --*
import json
import urllib
from urllib.request import Request, urlopen

In [148]:
# 주소에 geocoding 적용하는 함수를 작성.
def get_location(loc) :
    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(loc)
    
    # 주소 변환
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urlopen(request)
    res = response.getcode()
    
    if (res == 200) : # 응답이 정상적으로 완료되면 200을 return
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        # print(response_body)
        
        # 주소가 존재할 경우 total count == 1이 반환됨.
        if response_body['meta']['totalCount'] == 1 : 
        	# 위도, 경도 좌표를 받아와서 return해 줌.
            lat = response_body['addresses'][0]['y']
            lon = response_body['addresses'][0]['x']
            return (lon, lat)
        else :
            print('location not exist')
        
    else :
        print('ERROR')

In [149]:
# *-- Directions 5 활용 코드 --*

# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션)]
# trafast 실시간 빠른길
# tracomfort 실시간 편한길
# traoptimal 실시간 최적
# traavoidtoll 무료 우선
# traavoidcaronly 자동차 전용도로 회피 우선
option = 'traoptimal'

def get_optimal_route(start, goal, option=option ) :
    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    res = response.getcode()
    
    if (res == 200) :
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
            
    else :
        print('ERROR')

In [150]:
# 주소에서 위도, 경도 뽑고 dataframe 만드는 함수
def geo_df_coding(addresses):
    geo_list = []
    for address in addresses:
        geo_list.append(get_location(address))
        
    geo_df = pd.DataFrame(geo_list)
    geo_df.columns = ['longitude', 'latitude']
    geo_df = geo_df[['latitude', 'longitude']]
    
    return geo_df, geo_list

In [151]:
# 각 점 간도로상 거리, 시간 계산해주는 함수
def dist_time_mat(geo_df, geo_list):
    dist_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))
    time_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))

    option = 'traoptimal'
    for i in range(geo_df.shape[0]):
        for j in range(geo_df.shape[0]):
            if (i==j):
                dist_mat[i][j] = 0
                time_mat[i][j] = 0
            else:
                results = get_optimal_route(start = geo_list[i], goal = geo_list[j], option=option)

                dist_mat[i][j] = results['route']['traoptimal'][0]['summary']["distance"]
                time_mat[i][j] = results['route']['traoptimal'][0]['summary']["duration"]
                
    return dist_mat, time_mat

In [152]:
# 출발지로부터 각 점에 대한 거리, 시간 계산
def dist_time_start(start, geo_list):
    start_loc = get_location(start)
    
    dist_list = []
    time_list = []
    
    option = 'traoptimal'

    for i in range(len(geo_list)):
        results = get_optimal_route(start = start_loc, goal = geo_list[i], option=option)

        dist_list.append(results['route']['traoptimal'][0]['summary']["distance"])
        time_list.append(results['route']['traoptimal'][0]['summary']["duration"])
    
    return dist_list, time_list

### Clustering Algorithm

In [153]:
# k-means 적용
def k_means(geo_df, k):
    geo_df_k = geo_df.copy()
        
    geo_df_k = geo_df_k.astype('float')
    
    model = KMeans(n_clusters=k, init='k-means++')
    model.fit(geo_df)
    labels = model.predict(geo_df.values)
    geo_df_k['labels'] = labels
    
    return geo_df_k

In [154]:
# 각 그룹에서 출발점으로부터 모든 정점을 거쳤을 때 최단거리 산출하는 함수
# 1. 모든 정점 순서 경우 고려 (permutation)
# 2. 그 순서의 거리의 합을 따져서 최단거리 산출
def min_dist_start(geo_df_k, label, dist_list_start, dist_mat):
    '''
    Input : df, label(군집), dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 출발점으로부터 군집의 모든 정점을 거쳤을 때 최단거리
    '''
    
    # 모든 point 순서 고려 (permutation)
    permutations = list(itertools.permutations(geo_df_k[geo_df_k['labels']==label].index, sum(geo_df_k['labels']==label)))
    
    # 모든 경우 거리 합 계산 후 최소값 산출
    dist_sum_lst = []
    for i in range(len(permutations)):
        # 출발점(도봉구청)과 첫번째 점의 거리 초기화
        dist_sum =  dist_list_start[permutations[i][0]]
        # 순서대로 합 더해줌
        for j in range(len(permutations[0])-1):
            dist_sum += dist_mat[permutations[i][j], permutations[i][j+1]]
        dist_sum_lst.append(dist_sum)
    
    # 가장 작은 값 -> 최단거리
    return min(dist_sum_lst)

In [155]:
# 각 그룹에서 출발점으로부터 모든 정점을 거쳤을 때 최단거리인 경우 들린 순서 산출하는 함수
# 1. 모든 정점 순서 경우 고려 (permutation)
# 2. 그 순서의 거리의 합을 따져서 최단거리 산출
def min_dist_start_order(geo_df_k, label, dist_list_start, dist_mat):
    '''
    Input : df, label(군집), dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 출발점으로부터 군집의 모든 정점을 거쳤을 때 최단거리인 경우 들린 순서
    '''

    # 모든 point 순서 고려 (permutation)
    permutations = list(itertools.permutations(geo_df_k[geo_df_k['labels']==label].index, sum(geo_df_k['labels']==label)))
        
    # 모든 경우 거리 합 계산 후 최소값 산출     
    dist_sum_lst = []
    for i in range(len(permutations)):
        # 출발점(도봉구청)과 첫번째 점의 거리 초기화
        dist_sum =  dist_list_start[permutations[i][0]]
        # 순서대로 합 더해줌
        for j in range(len(permutations[0])-1):
            dist_sum += dist_mat[permutations[i][j], permutations[i][j+1]]
        dist_sum_lst.append(dist_sum)

    # 가장 작은 값 -> 최단거리순서
    return permutations[np.argmin(dist_sum_lst)]

In [156]:
# 모든 군집 최단거리 합 구하는 함수
def sum_mindist(geo_df_k, dist_list_start, dist_mat):
    '''
    Input : df, dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 모든 군집 최단거리 합 구하는 함수
    '''
    min_distance = []
    for i in geo_df_k['labels'].unique():
        min_distance.append(min_dist_start(geo_df_k, i, dist_list_start, dist_mat))

    return np.mean(min_distance)

In [157]:
# 각 군집의 출발점으로부터 최단거리끼리 차이의 분산
def start_var_diff(geo_df_k, dist_list_start, dist_mat):
    '''
    Input : df, dist_list_start(시작-점 거리), dist_mat(점-점 거리)
    Return : 각 군집의 출발점으로부터 최단거리 모든 조합 차이의 분산
    '''
    min_distance = []
    for i in geo_df_k['labels'].unique():
        min_distance.append(min_dist_start(geo_df_k, i, dist_list_start, dist_mat))

    # k combination 2 : 2개씩 차이의 분산
    combinations = list(itertools.combinations(geo_df_k['labels'].unique(), 2))
    
    diff_list = []
    for i, j in combinations:
        diff = np.abs(min_distance[i] - min_distance[j])
        diff_list.append(float(diff))

    return statistics.variance(diff_list)

In [158]:
# 다른 군집 point 중 해당 군집에 가장 가까운 point를 해당 군집으로 포함시키는 함수
# 중심점에서 가장 가까운 point 해당 군집으로 포함시킴
def near_point_start(geo_df_k, label):
    '''
    Input : df, 변경할 군집
    Return : 군집의 중심에서 다른 군집 point 중 가장 가까운 point 해당 군집으로 변경
    '''
    # 원본 dataframe 변경 방지
    geo_df_k_c = geo_df_k.copy()
    
    center = geo_df_k_c.groupby('labels')[['latitude', 'longitude']].mean().reset_index('labels')[['latitude', 'longitude']]

    # 군집이 i가 아닌 포인트 중 가장 가까운 포인트의 군집(labels)을 i로 변경
    idx = geo_df_k_c[geo_df_k_c['labels']!=label].index

    geo_df_k_c.loc[:, "distance_other_center"] = np.NAN
    for j in idx:
        geo_df_k_c.loc[j, "distance_other_center"] = haversine(geo_df_k_c.loc[j, ['latitude', 'longitude']], center.loc[label,['latitude', 'longitude']])

    if (geo_df_k_c['labels'].value_counts()[geo_df_k_c.loc[np.argmin(geo_df_k_c['distance_other_center']), 'labels']] != 1):
        geo_df_k_c.loc[np.argmin(geo_df_k_c['distance_other_center']), 'labels'] = label
        
    del geo_df_k_c['distance_other_center']
    
    return geo_df_k_c

In [159]:
# 모든 군집에 대해 point 추가 진행해보고 가장 분산이 작아지는 경우 선택하고 다음 loop 실행
def min_var_start_algorithm(geo_df_k, dist_list_start, dist_mat):
    '''
    Input : df
    Return : print - 변경될 떄 마다 cnt, var, return - 최종 df
    '''
    geo_df_k_c = geo_df_k.copy()

    cnt = 0
    # print("초기 Var : {}\n".format(start_var_diff(geo_df_k_c)))

    # 기존 군집화, 각 군집 변화시켜보고 var 비교 후 결정
    while(True):
        var_diff_list = []
        var_diff_list.append(start_var_diff(geo_df_k_c, dist_list_start, dist_mat))
        print("Var : {:.5}\n".format(start_var_diff(geo_df_k_c, dist_list_start, dist_mat)))
        
        k = len(geo_df_k_c['labels'].unique())
        for label in range(k):
            geo_df_k_1 = near_point_start(geo_df_k_c, label)
            var_diff_list.append(start_var_diff(geo_df_k_1, dist_list_start, dist_mat))

        if (np.min(var_diff_list) == var_diff_list[0]):
            break
        else:
            geo_df_k_c = near_point_start(geo_df_k_c, np.argmin(var_diff_list)-1)
            cnt += 1
            print("변경 횟수 : {}".format(cnt))
            # print("Var : {}\n".format(start_var_diff(geo_df_k_c)))
            
    print("총 변경 횟수 : {}".format(cnt))
    print("최종 결과 Var : {:.5}".format(start_var_diff(geo_df_k_c, dist_list_start, dist_mat)))
    return geo_df_k_c

## Main

### K-Means + postman algorithm vs postman algorithm for all cases

In [160]:
addresses = ['서울시 도봉구 방학로6길 25', 
           '서울시 도봉구 덕릉로63길 19',
           '서울시 도봉구 해등로 133',
           '서울시 도봉구 방학로3길 16',
           '서울시 도봉구 도봉로133길 42',
           '서울시 도봉구 덕릉로59나길 20',
           '서울시 도봉구 해등로16가길 32',
           '서울시 도봉구 우이천로34길 38',
           '서울시 도봉구 노해로 41길 9',
           '서울시 도봉구 도봉로 969',
           '서울특별시 도봉구 마들로 656']

# 출발지 : 도봉산역
start = "서울특별시 도봉구 도봉로 964-40"
start_loc = get_location(start)

geo_df, geo_list = geo_df_coding(addresses)

# 출발지로부터 각 점까지 거리 & 시간
dist_list_start, time_list_start = dist_time_start(start, geo_list)

# 각 점 서로 거리 & 시간
dist_mat, time_mat = dist_time_mat(geo_df, geo_list)

#### K-Means + postman Algorithm

In [161]:
start = timeit.default_timer()

# print("K = 3")
geo_df_3 = k_means(geo_df, 3)
geo_df_3_start = geo_df_3.copy()
geo_df_3_start = min_var_start_algorithm(geo_df_3_start, dist_list_start, dist_mat)

result_df_kmeans = pd.DataFrame()
result_df_kmeans.loc[0, 'var'] = start_var_diff(geo_df_3_start, dist_list_start, dist_mat)
result_df_kmeans.loc[0, 'sum_mindist'] = sum_mindist(geo_df_3_start, dist_list_start, dist_mat)

stop = timeit.default_timer()
result_df_kmeans.loc[0, 'runtime'] = stop-start


Var : 1.0825e+07

변경 횟수 : 1
Var : 1.6227e+06

변경 횟수 : 2
Var : 1.1392e+06

변경 횟수 : 3
Var : 1.0573e+06

총 변경 횟수 : 3
최종 결과 Var : 1.0573e+06


In [162]:
result_df_kmeans

,var,sum_mindist,runtime
0,1.057306e+06,6608.666667,0.849475


In [163]:
# # K=3 Experiment about run time
# time_list_3 = []
# for i in range(1000):
#     start = timeit.default_timer()

#     # print("K = 3")
#     geo_df_3 = k_means(geo_df, 3)
#     geo_df_3_start = geo_df_3.copy()
#     geo_df_3_start = min_var_start_algorithm(geo_df_3_start, dist_list_start, dist_mat)

#     stop = timeit.default_timer()
    
#     time_list_3.append(stop-start)
    
#     # print('Time: ', stop - start)  
# print(np.mean(time_list_3))

# print(np.mean(time_list_3))

#### All cases

In [186]:
def all_case(k, p):
    '''
    input : k(# of cluster), p(# of point)
    output : 군집배정하는 모든 경우의 수 (군집에 최소 1점씩은 있도록)
    '''        
    num_lst = []
    for cwr in itertools.combinations_with_replacement(np.arange(k), p):
        num_lst.append(cwr)
    
    label_list = []
    for i in num_lst:
        label_list.append(list(map(int, list(i))))

    label_list_pre = []
    if (k == 3):
        for i in range(len(label_list)):
            if (0 in label_list[i] and 1 in label_list[i] and 2 in label_list[i]):
                label_list_pre.append(label_list[i])
    elif (k == 4):
        for i in range(len(label_list)):
            if (0 in label_list[i] and 1 in label_list[i] and 2 in label_list[i] and 3 in label_list[i]):
                label_list_pre.append(label_list[i])
    elif (k == 5):
        for i in range(len(label_list)):
            if (0 in label_list[i] and 1 in label_list[i] and 2 in label_list[i] and 3 in label_list[i] and 4 in label_list[i]):
                label_list_pre.append(label_list[i])

    return label_list_pre


In [193]:
# K=3 Experiment about every case
label_list = all_case(k=3, p=11)
geo_df_all = geo_df.copy()
geo_df_all = geo_df_all.astype('float')

result_df = pd.DataFrame()

for i in range(len(label_list)):
    print("\ncount : {}".format(i))
    
    start = timeit.default_timer()
    
    geo_df_all.loc[:,'labels'] = label_list[i]

    geo_df_all = min_var_start_algorithm(geo_df_all, dist_list_start, dist_mat)

    result_df.loc[i, 'var'] = start_var_diff(geo_df_all, dist_list_start, dist_mat)
    result_df.loc[i, 'sum_mindist'] = sum_mindist(geo_df_all, dist_list_start, dist_mat)
    
    stop = timeit.default_timer()
    result_df.loc[i, 'runtime'] = stop-start

print("\n종료")



count : 0
Var : 1.6615e+07

변경 횟수 : 1
Var : 7.3281e+06

변경 횟수 : 2
Var : 5.8936e+06

총 변경 횟수 : 2
최종 결과 Var : 5.8936e+06

count : 1
Var : 5.0016e+06

변경 횟수 : 1
Var : 2.6374e+06

변경 횟수 : 2
Var : 1.9625e+06

변경 횟수 : 3
Var : 1.6089e+06

변경 횟수 : 4
Var : 1.0405e+06

총 변경 횟수 : 4
최종 결과 Var : 1.0405e+06

count : 2
Var : 7.1271e+06

변경 횟수 : 1
Var : 1.4567e+06

변경 횟수 : 2
Var : 2.9296e+05

변경 횟수 : 3
Var : 1.2129e+05

변경 횟수 : 4
Var : 4.9194e+04

총 변경 횟수 : 4
최종 결과 Var : 4.9194e+04

count : 3
Var : 4.8308e+06

변경 횟수 : 1
Var : 1.7063e+06

변경 횟수 : 2
Var : 8.0609e+05

변경 횟수 : 3
Var : 4.1821e+04

총 변경 횟수 : 3
최종 결과 Var : 4.1821e+04

count : 4
Var : 1.4567e+06

변경 횟수 : 1
Var : 2.9296e+05

변경 횟수 : 2
Var : 1.2129e+05

변경 횟수 : 3
Var : 4.9194e+04

총 변경 횟수 : 3
최종 결과 Var : 4.9194e+04

count : 5
Var : 1.7952e+06

변경 횟수 : 1
Var : 7.7778e+05

변경 횟수 : 2
Var : 2.9296e+05

변경 횟수 : 3
Var : 1.2129e+05

변경 횟수 : 4
Var : 4.9194e+04

총 변경 횟수 : 4
최종 결과 Var : 4.9194e+04

count : 6
Var : 7.1795e+06

변경 횟수 : 1
Var : 1.6826e+06


In [194]:
result_df

,var,sum_mindist,runtime
0,5.893610e+06,5314.333333,9.215723
1,1.040516e+06,7247.333333,1.954526
2,4.919433e+04,6470.666667,1.739147
3,4.182100e+04,6626.666667,0.465357
4,4.919433e+04,6470.666667,0.392133
5,4.919433e+04,6470.666667,1.168327
6,4.182100e+04,6626.666667,0.467998
7,1.934023e+05,6931.666667,0.229678
8,1.750933e+04,7801.333333,0.130821
9,7.718013e+05,7343.333333,0.133323


In [195]:
result_df_kmeans

,var,sum_mindist,runtime
0,1.057306e+06,6608.666667,0.849475


In [205]:
# 최적화된 결과가 총 두 가지 있는 듯ㄴ
result_df.loc[(result_df['var'] <= float(result_df_kmeans['var'])) & (result_df['sum_mindist'] <= float(result_df_kmeans['sum_mindist'])), :]

,var,sum_mindist,runtime
2,49194.333333,6470.666667,1.739147
4,49194.333333,6470.666667,0.392133
5,49194.333333,6470.666667,1.168327
39,936542.333333,6587.000000,0.216285
40,936542.333333,6587.000000,0.246144
